In [ ]:
from flask import Flask, request, jsonify
import joblib
import numpy as np
import pandas as pd

app = Flask(__name__)

# Load model and feature list
model = joblib.load('model.pkl')
feature_columns = joblib.load('feature_columns.pkl')

# Skill list to match original encoding
skill_list = ["Python", "Java", "C++", "SQL", "HTML", "CSS", "JavaScript", "Docker",
              "AWS", "Figma", "React", "Linux"]

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json

    # Extract inputs
    academic_year = data['academic_year']
    branch = data['branch']
    quiz_responses = data['quiz_responses']  # list of 10
    skills = data['skills']  # list of strings like ["Python", "Docker"]

    # Build the input row
    input_data = {}

    # One-hot encoding
    for col in feature_columns:
        input_data[col] = 0

    input_data[f'academic_year_{academic_year}'] = 1
    input_data[f'branch_{branch}'] = 1

    for i, answer in enumerate(quiz_responses, 1):
        input_data[f'quiz_q{i}_response_{answer}'] = 1

    for skill in skill_list:
        input_data[f'skill_{skill}'] = 1 if skill in skills else 0

    # Convert to DataFrame
    input_df = pd.DataFrame([input_data])

    # Ensure all columns are present
    input_df = input_df.reindex(columns=feature_columns, fill_value=0)

    # Predict
    pred = model.predict(input_df)

    # Normalize to 100
    pred = np.clip(pred, 0, None)
    pred = pred / pred.sum(axis=1, keepdims=True) * 100

    return jsonify({
        'visual_percent': round(pred[0][0], 2),
        'auditory_percent': round(pred[0][1], 2),
        'kinesthetic_percent': round(pred[0][2], 2)
    })

if __name__ == '__main__':
    app.run(port=5000)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [19/Jul/2025 18:42:10] "POST /predict HTTP/1.1" 200 -
